# Reproducing the results of the authors of the ResLogit paper [1]

We reproduce in this notebook the results of the authors of the ResLogit paper on the toy dataset *Pedestrian_Waittime.csv* (10 lines).

The toy dataset is available here: https://github.com/LiTrans/reslogit/blob/master/Pedestrian_Waittime.csv.

Code of the authors: https://github.com/LiTrans/reslogit.

In [ ]:
import os
import sys
import timeit
from pathlib import Path


sys.path.append("../../")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

from choice_learn.datasets.base import download_from_url, get_path, DATA_MODULE
from choice_learn.data import ChoiceDataset
from choice_learn.models.reslogit import ResLogit

import os

# Remove/Add GPU use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

The toy dataset is downloaded from the URL if it's not already present in the data directory.

In [ ]:

url = "https://raw.githubusercontent.com/LiTrans/reslogit/refs/heads/master/Pedestrian_Waittime.csv"
data_file_name = download_from_url(url)
full_path = get_path(data_file_name, module=DATA_MODULE)

In [ ]:
pedestrian_df = pd.read_csv(full_path)

# Rename columns corresponding to items features to be able to call ChoiceDataset.from_single_wide_df() 
pedestrian_df = pedestrian_df.rename(columns={"mode_active": "mode_1", "mode_Public Transit": "mode_2"})

print(f"Shape: {pedestrian_df.shape}")
print(f"Columns names: {pedestrian_df.columns}")
pedestrian_df.head()

The **output** we want to predict is the "category" (1 or 2).

The **inputs** are the columns: 
- "low LaneWidth",
- "high LaneWidth",
- "density",
- "mixed_TrafficCondition",
- "fully_AutomatedCondition",
- "Snowy",
- "One way",
- "Two way with median",
- "Night",
- "OverOneCar",
- "Age_30 - 39", 
- "age_over50", 
- "Gender_Female", 
- "mode_1", 
- "walk_to_work", 
- "mode_2".

This DataFrame is in the wide format, so we can use the 'from_single_wide_df' method.

In [ ]:
pedestrian_dataset = ChoiceDataset.from_single_wide_df(
    # The main DataFrame
    df=pedestrian_df,
    # The names of the items, will be used to find columns and organize them
    items_id=[1, 2],

    # Columns for shared_features_by_choice
    shared_features_columns=["low LaneWidth", "high LaneWidth", "density",
                             "mixed_TrafficCondition", "fully_AutomatedCondition",
                             "Snowy", "One way", "Two way with median", "Night",
                             "OverOneCar", "Age_30 - 39", "age_over50", "Gender_Female",
                             "walk_to_work",],
    
    # Columns for items_features_by_choice
    # They will be reconstructed as "prefix" + "item_id" + "delimiter"
    items_features_prefixes=["mode"],
    delimiter="_",

    # The column containing the choices
    choices_column="category",
    # How the choices are encoded: items_id means that the names of the choices correspond to the items_id list
    choice_format="items_id",
)

In [ ]:
pedestrian_dataset.summary()
print(f"\n\n{type(pedestrian_dataset)=}")
print(f"\n{np.shape(pedestrian_dataset.items_features_by_choice)=}")
print(f"{np.shape(pedestrian_dataset.shared_features_by_choice)=}")

## ResLogit modeling

In [ ]:
n_items = len(set(pedestrian_dataset.choices))
n_items_features = np.shape(pedestrian_dataset.items_features_by_choice)[3]
n_shared_features = np.shape(pedestrian_dataset.shared_features_by_choice)[2]
n_vars = n_items_features + n_shared_features
n_choices = len(np.unique(pedestrian_dataset.choices))

print(f"{n_items=}\n{n_items_features=}\n{n_shared_features=}\n{n_vars, n_choices=}")

In [ ]:
n_samples = len(pedestrian_dataset.choices)
# Slicing index for train and valid split
slice = np.floor(0.7 * n_samples).astype(int)
train_indexes = np.arange(0, slice)
test_indexes = np.arange(slice, n_samples)

train_dataset = pedestrian_dataset[train_indexes]
test_dataset = pedestrian_dataset[test_indexes]

start_time = timeit.default_timer()
model = ResLogit(intercept=None, n_layers=16, lr=0.001, epochs=1000, batch_size=64)

losses = model.fit(choice_dataset=train_dataset, val_dataset=test_dataset)
probas = model.predict_probas(test_dataset)

test_eval = tf.keras.losses.CategoricalCrossentropy(from_logits=False)(y_pred=probas, y_true=tf.one_hot(test_dataset.choices, n_items))
print(f"\n{losses=}\n{probas=}\n{test_eval=}\n")

end_time = timeit.default_timer()
print(f"Execution time for iteration: {end_time - start_time} seconds")

Once the model is estimated, we can look at the weights with the .trainable_weights argument:

In [ ]:
model.trainable_weights

In [ ]:
plt.plot(losses["train_loss"])
plt.xlabel("Epochs")
plt.ylabel("Negative log likelihood")
plt.title("Training of the ResLogit model")
plt.show()


We can easily acces the negative log likelihood value for the training dataset or another one using the .evaluate() method:

In [ ]:
print(f"Negative log likelihood: {model.evaluate(pedestrian_dataset)}")
print(f"Negative log likelihood multiplied by the size of the dataset: {len(pedestrian_dataset) * model.evaluate(pedestrian_dataset)}")
print("Average LogLikeliHood on test:", np.mean(test_eval))

### Same metrics as the authors of ResLogit

In [ ]:
# Predictions
print("Predictions:")
print(probas, "\n")
print(np.argmax(probas, axis=1), "\n")

# Confusion matrix
matrix = confusion_matrix(test_dataset.choices, np.argmax(probas, axis=1))
print(f"Confusion Matrix:\n {matrix}\n")

# Classification report
report = classification_report(test_dataset.choices, np.argmax(probas, axis=1))
print(f"Classification Report:\n {report}\n")


### References
[1] ResLogit: A residual neural network logit model for data-driven choice modelling, Wong, M.; Farooq, B (2021), Transportation Research Part C: Emerging Technologies 126\
(URL: https://doi.org/10.1016/j.trc.2021.103050)